In [59]:
import pandas as pd
import numpy as np

from datetime import datetime

%matplotlib inline
pd.set_option('display.max_rows', 500)

![Hi_there](img/CRISP_DM.png)

## Groupby apply function

- Note: continuing after the end of the subtopic 'Relational data model - defining a primary key'
- full data set too large to apply this function directly. Prepare a small test data set
- mapping some keys together and apply a function on it

In [60]:
pd_JH_data = pd.read_csv('../data/processed/COVID_relational_confirmed.csv', sep=';', parse_dates=[0])
pd_JH_data = pd_JH_data.sort_values('date', ascending = True).reset_index(drop = True).copy()
pd_JH_data.head()

,date,state,country,confirmed
0,2020-01-22,Alberta,Canada,0
1,2020-01-22,no,"Korea, South",1
2,2020-01-22,no,Kosovo,0
3,2020-01-22,no,Kuwait,0
4,2020-01-22,no,Kyrgyzstan,0


## Test data

- testing with two countries for some dates
- test data should be in a size where we can develop something and the possibility to review it

In [61]:
# get data according to country and the specified date
test_data = pd_JH_data[((pd_JH_data['country'] == 'US')|
                         (pd_JH_data['country'] == 'Germany'))&
                         (pd_JH_data['date'] > '2020-03-20')]

In [62]:
test_data.head()

,date,state,country,confirmed
15736,2020-03-21,no,Germany,22213
15777,2020-03-21,no,US,25600
16002,2020-03-22,no,Germany,24873
16042,2020-03-22,no,US,33276
16269,2020-03-23,no,Germany,29056


In [63]:
# group by the country and getting all info according to country and then aggregate
test_data.groupby(['country']).agg(np.max)

,date,state,confirmed
country,,,
Germany,2020-05-19,no,177778
US,2020-05-19,no,1528568


In [64]:
# %load ../src/features/build_features.py
# Linear regression models
import numpy as np
from sklearn import linear_model
reg = linear_model.LinearRegression(fit_intercept = True)

def get_doubling_time_via_regression(in_array):

    y = np.array(in_array)
    X = np.arange(-1,2).reshape(-1,1)

    assert len(in_array) == 3
    reg.fit(X,y)
    intercept = reg.intercept_
    slope = reg.coef_

    return intercept/slope

# Whenever the Python interpreter reads a source file, it does two things:
# it sets a few special variables like __name__, and then
# it executes all of the code found in the file.
# It's as if the interpreter inserts this at the top
# of your module when run as the main program.
# __name__ == "__main__"
#if __name__ == '__main__':
    #test_data = np.array([2,4,6])
    #result = get_doubling_time_via_regression(test_data)
    #print('The test slope is: ' + str(result))



In [65]:
# operate on the state as well
test_data.groupby(['state','country']).agg(np.max)

date  confirmed
state country                      
no    Germany 2020-05-19     177778
      US      2020-05-19    1528568

In [66]:
# test_data.groupby(['state', 'country']).apply(get_doubling_time_via_regression)

In [67]:
def rolling_reg(df_input, col = 'confirmed'):
    days_back = 3
    result = df_input[col].rolling(
             window = days_back,
             min_periods = days_back).apply(get_doubling_time_via_regression, raw = False)
    return result

In [68]:
test_data[['state', 'country', 'confirmed']].groupby(['state', 'country']).apply(rolling_reg, 'confirmed')

state  country       
no     Germany  15736           NaN
                16002           NaN
                16269      7.417994
                16535      7.142035
                16801      8.012983
                17067      6.954407
                17333      6.501919
                17599      7.390371
                17865     10.136671
                18130     13.541893
                18397     13.781393
                18663     13.140681
                18928     12.037271
                19196     12.735506
                19461     16.052694
                19726     21.372453
                19992     27.427355
                20258     27.511936
                20524     21.792179
                20790     21.495848
                21057     26.565108
                21322     36.198404
                21588     43.982990
                21854     49.423444
                22121     74.043747
                22385     56.424411
                22653     42.468318
      

In [69]:
test_data

,date,state,country,confirmed
15736,2020-03-21,no,Germany,22213
15777,2020-03-21,no,US,25600
16002,2020-03-22,no,Germany,24873
16042,2020-03-22,no,US,33276
16269,2020-03-23,no,Germany,29056
16309,2020-03-23,no,US,43843
16535,2020-03-24,no,Germany,32986
16575,2020-03-24,no,US,53736
16801,2020-03-25,no,Germany,37323
16840,2020-03-25,no,US,65778


In [70]:
# groupby function on the big data set
pd_DR_result = pd_JH_data[['state', 'country', 'confirmed']].groupby(['state', 'country']).apply(rolling_reg, 'confirmed').reset_index()
pd_DR_result.head()

,state,country,level_2,confirmed
0,Alberta,Canada,0,NaN
1,Alberta,Canada,465,NaN
2,Alberta,Canada,701,NaN
3,Alberta,Canada,966,NaN
4,Alberta,Canada,1263,NaN


In [71]:
pd_DR_result = pd_DR_result.rename(columns={'confirmed':'confirmed_DR',
                                           'level_2':'index'})
pd_DR_result.head()

,state,country,index,confirmed_DR
0,Alberta,Canada,0,NaN
1,Alberta,Canada,465,NaN
2,Alberta,Canada,701,NaN
3,Alberta,Canada,966,NaN
4,Alberta,Canada,1263,NaN


In [72]:
pd_JH_data.head()

,date,state,country,confirmed
0,2020-01-22,Alberta,Canada,0
1,2020-01-22,no,"Korea, South",1
2,2020-01-22,no,Kosovo,0
3,2020-01-22,no,Kuwait,0
4,2020-01-22,no,Kyrgyzstan,0


In [73]:
pd_JH_data = pd_JH_data.reset_index()
pd_JH_data.head()

,index,date,state,country,confirmed
0,0,2020-01-22,Alberta,Canada,0
1,1,2020-01-22,no,"Korea, South",1
2,2,2020-01-22,no,Kosovo,0
3,3,2020-01-22,no,Kuwait,0
4,4,2020-01-22,no,Kyrgyzstan,0


In [74]:
# merging the doubling rate to the large dataset
# Left join - right side matrix joined with left side
# During left-join, the index available on the left side is only merged 
# Joining or merging done 'on' index only
pd_result_larg = pd.merge(pd_JH_data, pd_DR_result[['index', 'confirmed_DR']], on = ['index'], how = 'left')
pd_result_larg

,index,date,state,country,confirmed,confirmed_DR
0,0,2020-01-22,Alberta,Canada,0,NaN
1,1,2020-01-22,no,"Korea, South",1,NaN
2,2,2020-01-22,no,Kosovo,0,NaN
3,3,2020-01-22,no,Kuwait,0,NaN
4,4,2020-01-22,no,Kyrgyzstan,0,NaN
...,...,...,...,...,...,...
31649,31649,2020-05-19,no,Barbados,90,88.666667
31650,31650,2020-05-19,no,Belarus,31508,32.913527
31651,31651,2020-05-19,no,Belgium,55791,217.390737
31652,31652,2020-05-19,no,Albania,949,631.777778


In [75]:
#pd_result_larg[pd_result_larg['country'] == 'Germany'].tail()

## Filtering the data with groupby apply

- adding new column step by step with new information to the large dataset

In [81]:
from scipy import signal

In [77]:
def savgol_filter(df_input, column ='confirmed', window = 5):
    window = 5
    degree = 1
    df_result = df_input
    
    filter_in = df_input[column].fillna(0) # Fill NA/NaN values using the specified method
    
    result = signal.savgol_filter(np.array(filter_in),
                                 window,
                                 degree)
    df_result[column+'_filtered'] = result
    return df_result

In [78]:
# groupping by the state and country
pd_filtered_result = pd_JH_data[['state', 'country', 'confirmed']].groupby(['state', 'country']).apply(savgol_filter).reset_index()

In [79]:
pd_filtered_result.tail()

,index,state,country,confirmed,confirmed_filtered
31649,31649,no,Barbados,90,89.8
31650,31650,no,Belarus,31508,31517.6
31651,31651,no,Belgium,55791,55825.4
31652,31652,no,Albania,949,954.6
31653,31653,no,Zimbabwe,46,46.4


In [80]:
# merge the columns 'index' and 'confirmed_filtered' from pd_filtered_result to pd_result_larg
pd_result_larg = pd.merge(pd_result_larg, pd_filtered_result[['index', 'confirmed_filtered']], on = ['index'], how = 'left')
pd_result_larg.head()

,index,date,state,country,confirmed,confirmed_DR,confirmed_filtered
0,0,2020-01-22,Alberta,Canada,0,NaN,0.0
1,1,2020-01-22,no,"Korea, South",1,NaN,0.8
2,2,2020-01-22,no,Kosovo,0,NaN,0.0
3,3,2020-01-22,no,Kuwait,0,NaN,0.0
4,4,2020-01-22,no,Kyrgyzstan,0,NaN,0.0


## Filtered doubling rate

-  doubling rate calculation done on the filtered data

In [117]:
pd_filtered_doubling = pd_result_larg[['state', 'country', 'confirmed_filtered']].groupby(['state', 'country']).apply(rolling_reg, 'confirmed_filtered').reset_index()
pd_filtered_doubling= pd_filtered_doubling.rename(columns={'confirmed_filtered':'confirmed_filtered_DR',
                             'level_2':'index'})
pd_filtered_doubling.tail()

,state,country,index,confirmed_filtered_DR
31649,no,Zimbabwe,30418,27.809524
31650,no,Zimbabwe,30654,23.803922
31651,no,Zimbabwe,30950,23.444444
31652,no,Zimbabwe,31186,29.200000
31653,no,Zimbabwe,31653,37.666667


In [119]:
pd_result_larg = pd.merge(pd_result_larg, pd_filtered_doubling[['index', 'confirmed_filtered_DR']], on = ['index'], how = 'left')
pd_result_larg.tail()

,index,date,state,country,confirmed,confirmed_DR,confirmed_filtered,confirmed_filtered_DR
31649,31649,2020-05-19,no,Barbados,90,88.666667,89.8,73.833333
31650,31650,2020-05-19,no,Belarus,31508,32.913527,31517.6,32.362549
31651,31651,2020-05-19,no,Belgium,55791,217.390737,55825.4,193.921089
31652,31652,2020-05-19,no,Albania,949,631.777778,954.6,116.851852
31653,31653,2020-05-19,no,Zimbabwe,46,45.333333,46.4,37.666667


In [122]:
mask = pd_result_larg['confirmed'] > 100
# False values will be assigned 'NaN'
pd_result_larg['confirmed_filtered_DR'] = pd_result_larg['confirmed_filtered_DR'].where(mask, other = np.NaN)

In [121]:
pd_result_larg[pd_result_larg['country'] == 'Germany'].tail()

,index,date,state,country,confirmed,confirmed_DR,confirmed_filtered,confirmed_filtered_DR
30365,30365,2020-05-15,no,Germany,175233,307.670485,175186.0,273.843531
30633,30633,2020-05-16,no,Germany,175752,274.967556,175676.6,309.920958
30897,30897,2020-05-17,no,Germany,176369,309.480047,176336.6,305.463352
31165,31165,2020-05-18,no,Germany,176551,441.111389,176925.5,282.349107
31430,31430,2020-05-19,no,Germany,177778,251.099125,177514.4,300.433860


In [125]:
pd_result_larg.to_csv('../data/processed/COVID_final_set.csv', sep=';',index=False)